In [1]:
import sys
print(sys.executable)

/Users/hugo/tensorflow-metal-test/bin/python


# Methods to be tested in this NLP model

1. Bag of words 
2. Tf idf 
3. word2vec 
4. Glove 
5. Bert 
6. Sbert etc... 
7. 4.5 sent2vec

In [80]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
df = pd.read_csv('../data/potential-talents - Aspiring human resources - seeking human resources.csv')

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


### Using a simple function to find keywords:

In [ ]:
# First option
def predict_fit(job_title):
    keywords = ["aspiring human resources", "seeking human resources"]
    for keyword in keywords:
        if keyword in job_title.lower():
            return 1  # Fit
    return 0  # Not fit

In [ ]:
df['fit'] = df['job_title'].apply(predict_fit)

In [ ]:
df.head()

In [ ]:
df['fit'].info()

In [ ]:
talents = (df['fit'] == 1).sum()
print(f"Talents based on key-word search': {talents}")

# Using Bag of Words:
* Not the best option for understanding semantics

In [ ]:
df = pd.read_csv('../data/potential-talents - Aspiring human resources - seeking human resources.csv')

In [ ]:
texts = df['job_title']

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
df_bow = pd.DataFrame(X.toarray(), columns=feature_names)

In [ ]:
merged_df = pd.concat([df, df_bow], axis=1)

In [ ]:
merged_df.head()

# Word2vec with straight probability

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

df['tokenized_titles'] = df['job_title'].apply(lambda x: x.lower().split())

In [ ]:
# Train a Word2Vec model
model = Word2Vec(df['tokenized_titles'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Define key phrases and tokenize
key_phrases = ["human resources", "human resources"]
key_phrases_tokenized = [phrase.split() for phrase in key_phrases]

In [ ]:
# Function to calculate average vector
def average_vector(words):
    return np.mean([model.wv[word] for word in words if word in model.wv], axis=0)

In [ ]:
# Calculate similarity and classify
threshold = 0.5  # Set your threshold
for phrase in key_phrases_tokenized:
    phrase_vector = average_vector(phrase)
    df['similarity_to_' + '_'.join(phrase)] = df['tokenized_titles'].apply(lambda x: cosine_similarity([average_vector(x)], [phrase_vector])[0][0])

In [ ]:
df['classification'] = df[[col for col in df if col.startswith('similarity_to')]].max(axis=1) >= threshold

In [ ]:
df

In [ ]:
df_fit = df[df['classification'] == True]

In [ ]:
df_fit['classification'].describe()

In [ ]:
df_fit

# Word2Vec with careful probability calculation
* This one is more porwerful at capturing semantics and sentence meanings but not the best

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Tokenized titles
df['tokenized_titles'] = df['job_title'].apply(lambda x: x.lower().split())

In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

# Training a Word2Vec model
model = Word2Vec(df['tokenized_titles'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Defining key phrases and tokenize
key_phrases = ["human resources"]
key_phrases_tokenized = [phrase.split() for phrase in key_phrases]

In [ ]:
# Function to calculate average vector
def average_vector(words):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if none of the words are in the model's vocabulary
        return np.zeros(model.vector_size)

# Check for exact matches
def contains_exact_phrase(title, phrases):
    return any(phrase in title for phrase in phrases)

In [ ]:
# Calculating similarity and classifying
threshold = 0.5
for phrase in key_phrases_tokenized:
    phrase_vector = average_vector(phrase)
    col_name = 'similarity_to_' + '_'.join(phrase)
    df[col_name] = df['job_title'].apply(lambda title: 1.0 if contains_exact_phrase(title.lower(), key_phrases) else cosine_similarity([average_vector(title.split())], [phrase_vector])[0][0])

In [ ]:
df['fit'] = df[[col for col in df if col.startswith('similarity_to')]].max(axis=1) >= threshold

In [ ]:
df

In [ ]:
df_fit = df[df['fit'] == True]

In [ ]:
df_fit['fit'].describe()

In [ ]:
df_fit

# Glove

In [ ]:
import gensim.downloader as api

# Load GloVe model (you can choose other versions as well)
glove_model = api.load("glove-wiki-gigaword-100")

In [ ]:
df['tokenized_titles'] = df['job_title'].apply(lambda x: x.lower().split())

In [ ]:
# Define key phrases and tokenize
key_phrases = ["aspiring human resources", "seeking human resources"]
key_phrases_tokenized = [phrase.split() for phrase in key_phrases]

In [ ]:
# Function to calculate average vector using GloVe
def average_vector(words):
    vectors = [glove_model[word] for word in words if word in glove_model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Assuming 100-dimensional vectors in GloVe

In [ ]:
# Calculate similarity and classify
threshold = 0.5
for phrase in key_phrases_tokenized:
    phrase_vector = average_vector(phrase)
    df['similarity_to_' + '_'.join(phrase)] = df['tokenized_titles'].apply(lambda x: cosine_similarity([average_vector(x)], [phrase_vector])[0][0])

In [ ]:
df['fit'] = df[[col for col in df if col.startswith('similarity_to')]].max(axis=1) >= threshold

In [ ]:
df

In [ ]:
df_fit = df[df['fit'] == True]

In [ ]:
df_fit['fit'].describe()

In [ ]:
df_fit

# Bert with Google Database

In [33]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [34]:
print(torch.__version__)

2.1.1


In [35]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8234, 0.0766, 0.6312],
        [0.5927, 0.2049, 0.4962],
        [0.8748, 0.9617, 0.8071],
        [0.8717, 0.5638, 0.4727],
        [0.3908, 0.8264, 0.1283]])


In [36]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [37]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [38]:
# Tokenization and Encoding for BERT
def bert_encode(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**encoded_input)
    # Flatten the tensor to 1-D
    return output.last_hidden_state.mean(dim=1).squeeze()

In [39]:
df['tokenized_titles'] = df['job_title'].apply(lambda x: x.lower())

In [69]:
# Define key phrases
key_phrases = ["aspiring human resources", "seeking human resources"]

In [41]:
# Compute BERT embeddings for key phrases
key_phrase_embeddings = [bert_encode(phrase) for phrase in key_phrases]

In [42]:
# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [59]:
# Calculate similarity and classify
threshold = 0.8
for idx, phrase in enumerate(key_phrases):
    phrase_embedding = key_phrase_embeddings[idx]
    df['similarity_to_' + phrase.replace(" ", "_")] = df['tokenized_titles'].apply(
        lambda x: cosine_similarity(bert_encode(x), phrase_embedding)
    )

In [60]:
df['fit'] = df[[col for col in df if col.startswith('similarity_to')]].max(axis=1) >= threshold

In [61]:
df

,id,job_title,location,connection,fit,tokenized_titles,similarity_to_aspiring_human_resources,similarity_to_seeking_human_resources
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,False,2019 c.t. bauer college of business graduate (...,0.587713,0.534470
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,False,native english teacher at epik (english progra...,0.546801,0.529103
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,False,people development coordinator at ryan,0.752289,0.738064
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,False,advisory board member at celal bayar university,0.430944,0.457767
...,...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,False,aspiring human resources manager | graduating ...,0.659241,0.619171
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,False,human resources generalist at loparex,0.681012,0.670784
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,False,business intelligence and analytics at travelers,0.645834,0.603013
102,103,Always set them up for Success,Greater Los Angeles Area,500+,False,always set them up for success,0.621722,0.601577


In [62]:
df_fit = df[df['fit'] == True]

In [63]:
df_fit['fit'].describe()

count       21
unique       1
top       True
freq        21
Name: fit, dtype: object

In [64]:
df_fit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 2 to 98
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      21 non-null     int64  
 1   job_title                               21 non-null     object 
 2   location                                21 non-null     object 
 3   connection                              21 non-null     object 
 4   fit                                     21 non-null     bool   
 5   tokenized_titles                        21 non-null     object 
 6   similarity_to_aspiring_human_resources  21 non-null     float64
 7   similarity_to_seeking_human_resources   21 non-null     float64
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 1.3+ KB


# Based on Bert, classify candidates based on highest probability

In [65]:
# Creating a copy of the sliced DataFrame
df_fit = df_fit.copy()

In [66]:
df_fit['total_similarity'] = df_fit['similarity_to_aspiring_human_resources'] + df_fit['similarity_to_seeking_human_resources']

In [67]:
df_sorted = df_fit.sort_values(by='total_similarity', ascending=False)

In [68]:
df_sorted

,id,job_title,location,connection,fit,tokenized_titles,similarity_to_aspiring_human_resources,similarity_to_seeking_human_resources,total_similarity
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,True,seeking human resources opportunities,0.823453,0.934551,1.758004
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,True,seeking human resources opportunities,0.823453,0.934551,1.758004
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,True,seeking human resources position,0.815410,0.929690,1.745099
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,True,aspiring human resources professional,0.902632,0.794786,1.697418
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,True,aspiring human resources professional,0.902632,0.794786,1.697418


# Bert based on other factors you deem necessary

In [85]:
# Load pre-trained model tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Tokenization and Encoding for BERT
def bert_encode(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**encoded_input)
    return output.last_hidden_state.mean(dim=1).squeeze()

# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

# Define key phrases (excluding 'Chicago, Illinois')
key_phrases = ["aspiring human resources", "seeking human resources"]

# Compute BERT embeddings for key phrases
key_phrase_embeddings = [bert_encode(phrase) for phrase in key_phrases]

# Calculate similarity for job titles
df['tokenized_titles'] = df['job_title'].apply(lambda x: x.lower())
for idx, phrase in enumerate(key_phrases):
    phrase_embedding = key_phrase_embeddings[idx]
    df['similarity_to_' + phrase.replace(" ", "_")] = df['tokenized_titles'].apply(
        lambda x: cosine_similarity(bert_encode(x), phrase_embedding)
    )

# Calculate similarity for 'Chicago, Illinois' with location
df['tokenized_locations'] = df['location'].apply(lambda x: x.lower())
chicago_embedding = bert_encode("Chicago, Illinois")
df['similarity_to_Chicago_Illinois'] = df['tokenized_locations'].apply(
    lambda x: cosine_similarity(bert_encode(x), chicago_embedding)
)

def connection_to_prob(connection):
    if connection.strip() == '500+':
        return 1.00
    else:
        # Remove any non-numeric characters before conversion
        numeric_connection = ''.join(filter(str.isdigit, connection))
        return min(int(numeric_connection) / 500, 1.00)

df['connection_prob'] = df['connection'].apply(connection_to_prob)

# Create the new columns
df['FIT'] = df['similarity_to_aspiring_human_resources'] + df['similarity_to_seeking_human_resources']
df['FIT_AND_CONNECTIONS'] = df['similarity_to_aspiring_human_resources'] + df['similarity_to_seeking_human_resources'] + df['similarity_to_Chicago_Illinois']
df['ALL_Ajusted_Location'] = df['FIT_AND_CONNECTIONS'] + df['connection_prob']

# Rank rows based on 'FIT' probability from highest to lowest
df_ranked = df.sort_values(by='FIT', ascending=False)
df_ranked

,id,job_title,location,connection,fit,tokenized_titles,similarity_to_aspiring_human_resources,similarity_to_seeking_human_resources,tokenized_locations,similarity_to_Chicago_Illinois,connection_prob,FIT,FIT_AND_CONNECTIONS,ALL_Ajusted_Location
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,seeking human resources opportunities,0.823453,0.934551,"chicago, illinois",1.000000,0.780,1.758004,2.758004,3.538004
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,seeking human resources opportunities,0.823453,0.934551,"chicago, illinois",1.000000,0.780,1.758004,2.758004,3.538004
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,seeking human resources position,0.815410,0.929690,"las vegas, nevada area",0.730831,0.096,1.745099,2.475930,2.571930
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,NaN,aspiring human resources professional,0.902632,0.794786,"kokomo, indiana area",0.686489,0.142,1.697418,2.383908,2.525908
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional,0.902632,0.794786,"raleigh-durham, north carolina area",0.640041,0.088,1.697418,2.337460,2.425460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,Human Resources Generalist at Schwan's,Amerika Birleşik Devletleri,500+,NaN,human resources generalist at schwan's,0.640211,0.615383,amerika birleşik devletleri,0.323220,1.000,1.255593,1.578813,2.578813
34,35,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member at celal bayar university,0.430944,0.457767,"i̇zmir, türkiye",0.474464,1.000,0.888711,1.363175,2.363175
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member at celal bayar university,0.430944,0.457767,"i̇zmir, türkiye",0.474464,1.000,0.888711,1.363175,2.363175
47,48,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member at celal bayar university,0.430944,0.457767,"i̇zmir, türkiye",0.474464,1.000,0.888711,1.363175,2.363175
